In [19]:
import os
import cv2
import pickle
import numpy as np
from PIL import Image


In [20]:
# get name for currect working directory 
cwd = os.getcwd()
# print(cwd)

# put training data in working directory 
# name "Images" (where I keep training faces) cwd\Images
img_dir = os.path.join(cwd,"Images")
# print(img_dir)

In [21]:
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')

recognizer = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []


# nested for loops to retreive files
for root,dirs,files in os.walk(img_dir):
    
    for file in files:
        if  file.endswith("JPG") or file.endswith("jpg"):
            
            # path of each photo
            path = os.path.join(root,file)
            
            # get names folders
            label = os.path.basename(os.path.dirname(path))
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id +=1
                
            id_ = label_ids[label]
#             print(label_ids)
            
            
            #some number for variable
            # verify this image, tune into NUMPY array
            # turn image gray
            pil_image = Image.open(path).convert("L") # grayscale
            image_array = np.array(pil_image,"uint8") # make into array
            
#             print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor =1.1, minNeighbors =5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
 


with open("labels.pickle",'wb') as f:
    pickle.dump(label_ids,f)
    
recognizer.train(x_train,np.array(y_labels))
recognizer.save("trainner.yml")